In [1]:
# Basic PDF RAG with chat history
# 1. Data ingestion
# 2. Data chunking
# 3. Vectorise and store the data using embeddings
# 4. Setup base prompt and history session arch.
# 5. Setup core chain which connects everything
# 6. Start testing!

In [2]:
# Data ingestion and splitting/chunking
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(chunk_size=800, chunk_overlap=50)
pdf_splitted = PyPDFLoader(file_path="data/EduTrack_FAQ_assignment.pdf").load_and_split(text_splitter=splitter)

In [3]:
pdf_splitted

[Document(metadata={'producer': 'ReportLab PDF Library - www.reportlab.com', 'creator': '(unspecified)', 'creationdate': '2025-06-05T12:53:08+00:00', 'author': '(anonymous)', 'keywords': '', 'moddate': '2025-06-05T12:53:08+00:00', 'subject': '(unspecified)', 'title': '(anonymous)', 'trapped': '/False', 'source': 'data/EduTrack_FAQ_assignment.pdf', 'total_pages': 2, 'page': 0, 'page_label': '1'}, page_content='EduTrack – Frequently Asked Questions\nQ1: What is EduTrack used for?\nA1: EduTrack helps educational institutions monitor student engagement, analyze learning\nbehavior, and proactively support at-risk learners through data-driven insights.\nQ2: Which platforms does EduTrack integrate with?\nA2: EduTrack integrates seamlessly with LMS platforms such as Moodle, Canvas,\nBlackboard, Google Classroom, and can be extended to custom LMS solutions via API.\nQ3: What types of student data does EduTrack analyze?\nA3: It analyzes logins, session duration, assignment submission patterns, q

In [4]:
# Vectorise and store
from dotenv import load_dotenv
import os
from langchain_ollama.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS

load_dotenv()
os.environ['HF_TOKEN'] = os.getenv('HF_TOKEN')

embedding = OllamaEmbeddings(model="mxbai-embed-large")

vector_db = FAISS.from_documents(embedding=embedding, documents=pdf_splitted)

In [5]:
# Simple query to check if result are good
vector_db.similarity_search("What format can I access reports in?", k=1)

[Document(id='cdf5702d-c9f2-4571-af51-ec41a8e8e581', metadata={'producer': 'ReportLab PDF Library - www.reportlab.com', 'creator': '(unspecified)', 'creationdate': '2025-06-05T12:53:08+00:00', 'author': '(anonymous)', 'keywords': '', 'moddate': '2025-06-05T12:53:08+00:00', 'subject': '(unspecified)', 'title': '(anonymous)', 'trapped': '/False', 'source': 'data/EduTrack_FAQ_assignment.pdf', 'total_pages': 2, 'page': 1, 'page_label': '2'}, page_content='A10: Admins can track student performance trends across courses, compare program\neffectiveness, and identify gaps in instructor engagement.\nQ11: Is there a self-hosted version of EduTrack?\nA11: Yes. EduTrack offers both cloud-hosted SaaS and on-premise deployments for\ninstitutions with specific data residency needs.\nQ12: What formats are reports available in?\nA12: Reports can be downloaded in PDF, CSV, or Excel, and shared via secure URLs or\nembedded into LMS dashboards.\nQ13: How often is data updated in the system?\nA13: Data is 

In [6]:
# 4. Setup base prompt temaplate
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

template = ChatPromptTemplate.from_messages([
    ("system","You are a funny assistant who answers questions on the basis of the provided context: {context}"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "Here's the question: {input}")
])

In [7]:
# 5. Setup session and messages chain
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.messages import trim_messages
from langchain_core.messages.utils import count_tokens_approximately
from langchain_community.chat_message_histories import ChatMessageHistory 
from langchain_core.runnables import RunnablePassthrough
from operator import itemgetter
from langchain_ollama import ChatOllama
from langchain_core.runnables import RunnableWithMessageHistory
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain_core.runnables import RunnableLambda

llm = ChatOllama(model="gemma3:4b")

session = {}

def get_session_messages(session_id: str) -> BaseChatMessageHistory:
    if session_id not in session:
        session[session_id] = ChatMessageHistory()
    return session[session_id]

trimmer = trim_messages(max_tokens=800, strategy="last", include_system=True, token_counter=count_tokens_approximately)

# Create retriever
retriever = vector_db.as_retriever(k=1)

# Create the RAG chain with proper history handling
def create_rag_chain():
    # Retrieve documents
    retrieve_docs = itemgetter("input") | retriever
    
    # Format the chain
    rag_chain = (
        RunnablePassthrough.assign(
            context=retrieve_docs,
            chat_history=lambda x: trimmer.invoke(x.get("chat_history", []))
        )
        | template
        | llm
    )
    
    return rag_chain

# Create the main chain
rag_chain = create_rag_chain()

# Wrap with message history
chat_with_history_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history=get_session_messages,
    input_messages_key="input",
    history_messages_key="chat_history",
)

In [8]:
# 6. Invoke the chain and test out!
from langchain_core.messages import HumanMessage
config = {'configurable': {'session_id': 1}}

res = chat_with_history_chain.invoke({"input": "Can I self host edutrack?"}, config=config)
res

AIMessage(content='Absolutely! According to the documentation, EduTrack offers both cloud-hosted SaaS and on-premise deployments. So yes, you can definitely self-host it! 😊', additional_kwargs={}, response_metadata={'model': 'gemma3:4b', 'created_at': '2025-07-26T05:09:54.419176Z', 'done': True, 'done_reason': 'stop', 'total_duration': 2434680333, 'load_duration': 59131833, 'prompt_eval_count': 1535, 'prompt_eval_duration': 1752503834, 'eval_count': 35, 'eval_duration': 619006041, 'model_name': 'gemma3:4b'}, id='run--c6fce979-986d-44e8-8c7d-ff1f22b81485-0', usage_metadata={'input_tokens': 1535, 'output_tokens': 35, 'total_tokens': 1570})

In [9]:
res = chat_with_history_chain.invoke({"input": "My name is Anirudh"}, config=config)
res

AIMessage(content='Okay, Anirudh! What can I help you with today? 😊', additional_kwargs={}, response_metadata={'model': 'gemma3:4b', 'created_at': '2025-07-26T05:09:56.401646Z', 'done': True, 'done_reason': 'stop', 'total_duration': 1923296125, 'load_duration': 51656833, 'prompt_eval_count': 1485, 'prompt_eval_duration': 1590853167, 'eval_count': 16, 'eval_duration': 271720500, 'model_name': 'gemma3:4b'}, id='run--909f26f2-3024-4372-9412-27d8ce71cc49-0', usage_metadata={'input_tokens': 1485, 'output_tokens': 16, 'total_tokens': 1501})

In [11]:
res = chat_with_history_chain.invoke({"input": "What is my name?"}, config=config)
res

AIMessage(content='Okay, okay, let’s get this straight! According to the provided documents, your name is Anirudh! \n\nSeriously, I’m just repeating it because you asked! 😄', additional_kwargs={}, response_metadata={'model': 'gemma3:4b', 'created_at': '2025-07-26T05:10:13.661194Z', 'done': True, 'done_reason': 'stop', 'total_duration': 2569899791, 'load_duration': 58996916, 'prompt_eval_count': 1550, 'prompt_eval_duration': 1793162417, 'eval_count': 40, 'eval_duration': 699394208, 'model_name': 'gemma3:4b'}, id='run--278a1304-dae8-43e2-87e2-77387d84f95c-0', usage_metadata={'input_tokens': 1550, 'output_tokens': 40, 'total_tokens': 1590})

In [12]:
res = chat_with_history_chain.invoke({"input": "Who will take care of my training?"}, config=config)
res

AIMessage(content='According to the documents, new clients receive onboarding workshops, video tutorials, live Q&A sessions, and access to EduTrack’s support portal. So, you’ll be getting a pretty comprehensive team to help you get up and running! 😊', additional_kwargs={}, response_metadata={'model': 'gemma3:4b', 'created_at': '2025-07-26T05:10:41.659941Z', 'done': True, 'done_reason': 'stop', 'total_duration': 2668907291, 'load_duration': 67791041, 'prompt_eval_count': 1508, 'prompt_eval_duration': 1718932083, 'eval_count': 51, 'eval_duration': 859624083, 'model_name': 'gemma3:4b'}, id='run--a01648b0-4a06-4236-86db-10f0c4da0b74-0', usage_metadata={'input_tokens': 1508, 'output_tokens': 51, 'total_tokens': 1559})